# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import requests
import xlsxwriter
import statistics
from secrets import IEX_CLOUD_API_TOKEN


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
api_token = IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "AAPL"
api_url = f"https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={api_token}"
data = requests.get(api_url).json()
data

[{'beta': 1.3081350430908174,
  'totalCash': 62482000000,
  'currentDebt': 124963000000,
  'revenue': 383933000000,
  'grossProfit': 166816000000,
  'totalRevenue': 383933000000,
  'EBITDA': 123957000000,
  'revenuePerShare': 24.56,
  'revenuePerEmployee': 2611789.12,
  'debtToEquity': 5.558582473371603,
  'profitMargin': 0.2468138972164414,
  'enterpriseValue': 2782944942320,
  'enterpriseValueToRevenue': 7.25,
  'priceToSales': 7.13,
  'priceToBook': 45.4,
  'forwardPERatio': 29.33663254394669,
  'pegRatio': -16.787618956162493,
  'peHigh': 14.643369026365589,
  'peLow': 9.31113569934022,
  'week52highDate': '2023-07-19',
  'week52lowDate': '2023-01-03',
  'putCallRatio': 0.5166976347109928,
  'companyName': 'Apple Inc',
  'marketcap': 2736146942320,
  'week52high': 197.7,
  'week52low': 123.48,
  'week52highSplitAdjustOnly': 198.23,
  'week52highDateSplitAdjustOnly': '2023-07-19',
  'week52lowSplitAdjustOnly': 124.17,
  'week52lowDateSplitAdjustOnly': '2023-01-03',
  'week52change':

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
one_year_return = data[0]["year1ChangePercent"]
print(one_year_return)


0.17125751067791173


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

Since batch api call limit is 2000 in iexcloud, I do not prefer using chunks for stocks list.

In [5]:
stocks_string = ",".join(list(stocks["Ticker"]))

#batch api call for quote and advanced_stats
api_call_url = f"https://api.iex.cloud/v1/data/core/quote,advanced_stats/{stocks_string}?token={api_token}"
json_data = requests.get(api_call_url).json()

#received json file lists only the quote data of the symbols in the first 505 entry and the stats data in the second 505 entry
#we merge them in one dataset
quote_data = json_data[:505]
stats_data = json_data[505:]
merged_data = []
for i in range(len(quote_data)):
    quote_entry = quote_data[i]
    stats_entry = stats_data[i]
    
    merged_entry = {**quote_entry, **stats_entry}
    merged_data.append(merged_entry)
merged_data

[{'avgTotalVolume': 1915716,
  'calculationPrice': 'close',
  'change': 1.03,
  'changePercent': 0.00897,
  'close': 115.91,
  'closeSource': 'official',
  'closeTime': 1694808029624,
  'companyName': 'Agilent Technologies Inc.',
  'currency': 'USD',
  'delayedPrice': 115.9,
  'delayedPriceTime': 1694807984816,
  'extendedChange': -0.87,
  'extendedChangePercent': -0.00751,
  'extendedPrice': 115.04,
  'extendedPriceTime': 1694821031742,
  'high': 121.92,
  'highSource': '15 minute delayed price',
  'highTime': 1694807999999,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 116,
  'iexCloseTime': 1694807996644,
  'iexLastUpdated': 1694808545068,
  'iexMarketPercent': 0.03286737687433922,
  'iexOpen': 114.395,
  'iexOpenTime': 1694784615011,
  'iexRealtimePrice': 117.24,
  'iexRealtimeSize': 1,
  'iexVolume': 226444,
  'lastTradeTime': 1694807999844,
  'latestPrice': 115.91,
  'latestSource': 'Close',
  'latestTime': 'September 15, 2023',
  'la

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

for x, symbol in enumerate(stocks["Ticker"]):
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            merged_data[x]['latestPrice'],
            merged_data[x]['year1ChangePercent'],
            "N/A"
        ],
        index=my_columns),
        ignore_index=True
    )

pd.set_option('display.max_rows', None) #to show the all rows of the dataframe
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,115.910,-0.101581,N/A
1,AAL,13.310,-0.032,N/A
2,AAP,59.820,-0.63174,N/A
3,AAPL,175.010,0.171258,N/A
4,ABBV,152.120,0.0987876,N/A
5,ABC,179.980,0.238434,N/A
6,ABMD,381.020,None,N/A
7,ABT,101.740,-0.00297129,N/A
8,ACN,312.530,0.164699,N/A
9,ADBE,528.890,0.76591,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
#First converting "None" values to NaN to pick the 50 highest-momentum stocks
final_dataframe["One-Year Price Return"] = pd.to_numeric(final_dataframe["One-Year Price Return"], errors="coerce")

highest_momentum_50 = final_dataframe.nlargest(50, "One-Year Price Return", keep="all")
#Resetting the indexes
highest_momentum_50.reset_index(drop=True, inplace=True)
highest_momentum_50



,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,439.00,2.328297,N/A
1,LB,79.92,2.281287,N/A
2,FTI,20.86,1.313844,N/A
3,GE,115.49,1.218930,N/A
4,PHM,76.27,0.965864,N/A
5,RCL,97.42,0.931404,N/A
6,LLY,575.66,0.883792,N/A
7,ADBE,528.89,0.765910,N/A
8,AVGO,851.68,0.754308,N/A
9,BKNG,3160.15,0.684111,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [ ]:
def portfolio_size():
    portfolio_size = input("Enter the value of your portfolio: ")

    while True:
        try:
            val = float(portfolio_size)
            return val
        except ValueError:
            print("That's not a number \nPlease try again:")
            portfolio_size = input("Enter the value of your portfolio: ")
            continue
portfolio = portfolio_size()


In [ ]:
def numberOfSharesToBuy(portfolio, data_frame):
    portfolio_size_perStock = portfolio/len(data_frame)
    for i in range(50):     
        data_frame.loc[i, "Number of Shares to Buy"] = math.floor(portfolio_size_perStock/data_frame.loc[i, "Price"])
    return data_frame

numberOfSharesToBuy(portfolio, highest_momentum_50)


# Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
hqm_columns = ['Ticker', 
               'Price', 
               '1-year price returns',
               '1-year return percentile',
               '6-month price returns',
               '6-month return percentile',
               '3-month price returns', 
               '3-month return percentile', 
               '1-month price returns', 
               '1-month return percentile', 
               'Number of Shares to Buy']
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for x, symbol in enumerate(stocks["Ticker"]):
    hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            merged_data[x]['latestPrice'], 
            merged_data[x]['year1ChangePercent'],
            "NaN",
            merged_data[x]['month6ChangePercent'],
            "NaN",
            merged_data[x]['month3ChangePercent'],
            "NaN",
            merged_data[x]['month1ChangePercent'],
            "NaN",
            "NaN"
        ],
        index=hqm_columns),
        ignore_index=True
    )
    
#converting non numeric values to NaN, but excluding the Ticker column with [1:] since it contains only names 
hqm_dataframe[hqm_columns[1:]] = hqm_dataframe[hqm_columns[1:]].apply(lambda x: pd.to_numeric(x, errors="coerce"))
hqm_dataframe



## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
time_periods = [
    "1-year",
    "6-month",
    "3-month",
    "1-month"
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f"{time_period} return percentile"] = stats.percentileofscore(
            hqm_dataframe[f"{time_period} price returns"], 
            hqm_dataframe.loc[row, f"{time_period} price returns"]) / 100
hqm_dataframe

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
for row in hqm_dataframe.index:
    hqm_list = []
    for time_period in time_periods:
        hqm_list.append(hqm_dataframe.loc[row, f"{time_period} return percentile"])
    hqm_score = statistics.mean(hqm_list)
    hqm_dataframe.loc[row, "HQM Score"] = hqm_score
hqm_dataframe


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
best_50_hqm = hqm_dataframe.nlargest(50, ["HQM Score"], keep="all")
best_50_hqm.reset_index(drop=True, inplace=True)
best_50_hqm


## Calculating the Number of Shares to Buy

We'll use the `portfolio_size` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
hqm_portfolio = portfolio_size()

In [ ]:
numberOfSharesToBuy(hqm_portfolio, best_50_hqm)


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine="xlsxwriter")
best_50_hqm.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A' : ["Ticker", string_template],
    'B' : ["Price", dollar_template],
    'C' : ["1-year price returns", percent_template],
    'D' : ["1-year return percentile", percent_template],
    'E' : ["6-month price returns", percent_template],
    'F' : ["6-month return percentile", percent_template],
    'G' : ["3-month price returns", percent_template],
    'H' : ["3-month return percentile", percent_template],
    'I' : ["1-month price returns", percent_template],
    'J' : ["1-month return percentile", percent_template],
    'K' : ["Number of Shares to Buy", integer_template],
    'L' : ["HQM Score", percent_template]
    }
for column in column_formats:
    writer.sheets["Momentum Strategy"].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f'{column}1', column_formats[column][0], string_template)


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()